Convolutional Neural Network trained on CIFAR10 Data

Author: Brady Gho

Created on 7/15/2024, last editted on 7/15/2024

In [ ]:
# Meta Parameters

LEARNING_RATE = 0.03
EPOCHS=50
BATCH_SIZE=64
LAMBDA=0.0001

In [ ]:
# Loading Data
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    # add more transformations here
])

traindata = torchvision.datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_set= torchvision.datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

val_size = int(len(traindata)*0.2)
train_size = int(len(traindata)*0.8)

train_set, val_set = torch.utils.data.random_split(traindata, [train_size, val_size])


train_dataloader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size = BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Defining the Network

import torch
import torch.nn as nn

class BradyCNN(nn.Module):
  def __init__(self):
    super(BradyCNN, self).__init__()
    # cl = convolutional layer, fc = fully connected layer
    self.cl1 = nn.Conv2d(3, 8, kernel_size=5, stride=1, padding=2)
    self.cl2 = nn.Conv2d(8, 15, kernel_size=5, stride=1)
    self.cl3 = nn.Conv2d(15, 27, kernel_size=3, stride=1)
    # Add conv layers
    # Change kernel size
    # mess around with dimension

    self.fc1 = nn.Linear(27*4*4, 280)
    self.fc2 = nn.Linear(280, 49)
    self.fc3 = nn.Linear(49, 10)
    self.dropout = nn.Dropout(p=0.4)
    self.ReLU = nn.ReLU()
    self.maxPool = nn.MaxPool2d(kernel_size=3, stride=2)
    # kernel size = 3, stride = 2
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
      x = self.ReLU(self.cl1(x))  # 32 x 32
      x = self.maxPool(x)         # 16 x 16
      x = self.ReLU(self.cl2(x))  # 12 x 12
      x = self.ReLU(self.cl3(x))  # 10x10
      x = self.maxPool(x)         # 5 x 5
      x = x.view(x.size(0), -1)
      x = self.ReLU(self.fc1(x))
      x = self.dropout(x)
      x = self.ReLU(self.fc2(x))
      x = self.dropout(x)
      x = self.softmax(self.fc3(x))
      return x

def eval_model(model, dataloader, criterion):

  model.eval()

  total_loss = 0
  total_correct = 0

  with torch.no_grad():
    for inputs, answers in dataloader:
      predictions = model(inputs)
      loss = criterion(predictions, answers)
      total_loss += loss.sum()

      predictions = torch.argmax(predictions, dim=1)
      correct = (predictions == answers).float().sum()
      total_correct+=correct

  return total_loss / len(dataloader), round(100 * float(total_correct / (len(dataloader)*BATCH_SIZE)), 4)

def loss_function

In [ ]:
CNN = BradyCNN()

criterion = nn.CrossEntropyLoss()

train_loss_history = []
val_loss_history = []
val_accuracy_history = []

initial_test_loss, initial_test_accuracy = eval_model(CNN, test_dataloader, criterion)

In [ ]:
# Training the model

import torch.optim as optim
import numpy

optimizer = optim.Adagrad(CNN.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):

  CNN.train()
  for inputs, answers in train_dataloader:
    optimizer.zero_grad()
    predictions = CNN(inputs)
    loss = criterion(predictions, answers)

    L2_reg = 0
    for param in CNN.parameters():
      L2_reg += torch.sum(torch.square(param))
    loss+=LAMBDA*L2_reg

    train_loss_history.append(loss.item())

    loss.backward()
    optimizer.step()

  eval_loss, eval_accuracy = eval_model(CNN, val_dataloader, criterion)
  print(f"Epoch {epoch+1}: Loss = {eval_loss}, Accuracy = {eval_accuracy}%")

  val_loss_history.append(eval_loss)
  val_accuracy_history.append(eval_accuracy)

Epoch 1: Loss = 2.302597999572754, Accuracy = 9.7333%
Epoch 2: Loss = 2.223776340484619, Accuracy = 20.5414%
Epoch 3: Loss = 2.1835389137268066, Accuracy = 26.2838%
Epoch 4: Loss = 2.1670663356781006, Accuracy = 27.9658%
Epoch 5: Loss = 2.1701560020446777, Accuracy = 27.1696%
Epoch 6: Loss = 2.148444890975952, Accuracy = 29.996%
Epoch 7: Loss = 2.1402015686035156, Accuracy = 31.0311%
Epoch 8: Loss = 2.132145881652832, Accuracy = 31.6979%
Epoch 9: Loss = 2.1329362392425537, Accuracy = 31.5585%
Epoch 10: Loss = 2.1210315227508545, Accuracy = 33.1011%
Epoch 11: Loss = 2.1202995777130127, Accuracy = 32.8125%
Epoch 12: Loss = 2.123422145843506, Accuracy = 32.5637%
Epoch 13: Loss = 2.115668535232544, Accuracy = 33.2404%
Epoch 14: Loss = 2.110661745071411, Accuracy = 33.8177%
Epoch 15: Loss = 2.108877658843994, Accuracy = 34.0864%
Epoch 16: Loss = 2.105620861053467, Accuracy = 34.4347%


KeyboardInterrupt: 

In [ ]:
final_test_loss, final_test_accuracy = eval_model(CNN, test_dataloader, criterion)

print(f"Initial Test Results: Loss = {initial_test_loss}, Accuracy = {initial_test_accuracy}%")
print(f"Final Test Results: Loss = {final_test_loss}, Accuracy = {final_test_accuracy}%")

In [ ]:
# Plots

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(train_loss_history, label="Train Loss History")
plt.title("Train Loss History")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(val_loss_history, label="Validation Loss History")
plt.title("Validation Loss History")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(val_accuracy_history, label="Validation Accuracy History")
plt.title("Validation Accuracy History")
plt.xlabel("Epochs")
plt.ylabel("Percent Accuracy")
plt.show()
